https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings

In [1]:
from collections import Counter
import gc
from pathlib import Path
import re
import time

import numpy as np
import pandas as pd
import gensim

from tqdm import tqdm
tqdm.pandas()

In [2]:
data_dir = Path.home() / 'Desktop/kaggle/data/quora'

train_all = pd.read_csv(data_dir / 'train.csv.zip')
test_all = pd.read_csv(data_dir / 'test.csv.zip')

print("Train shape : ", train_all.shape)
print("Test shape : ", test_all.shape)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


In [3]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = Counter()
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            vocab[word] += 1
    return vocab

In [4]:
sentences = train_all["question_text"].progress_apply(lambda x: x.split()).values
vocab1 = build_vocab(sentences)

100%|██████████| 1306122/1306122 [00:06<00:00, 208512.89it/s]


In [5]:
vocab1.most_common(5)

[('the', 653948),
 ('What', 417802),
 ('to', 403183),
 ('a', 402682),
 ('in', 363131)]

In [7]:
news_path = data_dir / 'GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embeddings_index = gensim.models.KeyedVectors.load_word2vec_format(news_path, binary=True)

In [ ]:
def check_coverage(vocab, embeddings_index):
    covered_vocab = set()
    oov = Counter()
    covered_count = 0
    all_count = 0
    for word, count in tqdm(vocab.items()):
        all_count += count
        if word in embeddings_index:
            covered_vocab.add(word)
            covered_count += count
        else:
            oov[word] = count

    print('Found embeddings for {:.2%} of vocab'.format(len(covered_vocab) / len(vocab)))
    print('Found embeddings for  {:.2%} of all words'.format(covered_count / all_count))
    return oov

In [ ]:
oov = check_coverage(vocab1, embeddings_index)

In [ ]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

cnt = 0
for word in embeddings_index.vocab:
    if len(word) not in [1]:
        continue
    if '#' in word:
        continue
    if not is_ascii(word):
        continue
    if not re.fullmatch(r'[^a-zA-Z0-9]+', word):
        continue
    print(word, end=' ')
    cnt += 1
    if cnt > 100:
        break        

In [ ]:
punct_covered = re.compile(r'(^|\b|\s)([#=+*%_`>^~@$&])($|\b|\s)')

for s in ['a+b', 'aa+bb+c', 'a++b']:
    print(punct_covered.sub(lambda m: ' {} '.format(m.group(2)), s))

In [ ]:
punct_unknown = re.compile(r'(^|\b|\s)[?!.,:;"\'/|(){}\[\]\\-]($|\b|\s)')

for s in ['{b}', 'a\\b', 'a[b]c(d)e!', '"The"', 'it, ']:
    print(punct_unknown.sub(' ^ ', s))

In [ ]:
punct_numbers = re.compile(r'([0-9]{2,})')

for s in ['1', '11', '111', '1111']:
    print(punct_numbers.sub(lambda m: '#' * len(m.group(1)), s))

In [ ]:
rewrite_dict = {
    "What's": 'What is',
    "I'm": 'I am',
    "Shouldn't": 'Should not',
    "Wouldn't": 'Would not',
    "Couldn't": 'Could not',
    "Wasn't": 'Was not',
    "Aren't": 'Are not',
    "don't": 'do not',
    "can't": 'cannot',
    'colour':'color',
    'centre':'center',
    'didnt':'did not',
    "didn't": 'did not',
    'doesnt':'does not',
    "doesn't": 'does not',
    'isnt':'is not',
    "isn't": 'is not',
    'shouldnt':'should not',
    "shouldn't": 'should not',
    'behaviour': 'behavior',
    'favourite':'favorite',
    'travelling':'traveling',
    'counselling':'counseling',
    'theatre':'theater',
    'cancelled':'canceled',
    'labour':'labor',
    'organisation':'organization',
    'programme': 'program',
    'wwii':'world war 2',
    'citicise':'criticize',
    'instagram': 'social medium',
    'whatsapp': 'social medium',
    'snapchat': 'social medium',
    'Snapchat': 'social medium',
}

rewrite_re = re.compile('(%s)' % '|'.join(rewrite_dict.keys()))

In [ ]:
def clean_text(x):
    x = str(x)
    x = x.replace("’", "'")
    x = rewrite_re.sub(lambda m: rewrite_dict[m.group(0)], x)
    x = punct_covered.sub(lambda m: ' {} '.format(m.group(2)), x)
    x = punct_unknown.sub(' ^ ', x)
    x = punct_numbers.sub(lambda m: '#' * len(m.group(1)), x)
    return x

In [ ]:
cnt = 0
for k, v in sorted(oov.items(), key=lambda x: -x[1]):
    if re.fullmatch(r'[a-zA-Z]+[.,?]?', k):
        continue
    print('{} => {}'.format(k, v))
    cnt += 1
    if cnt > 20:
        break

In [ ]:
cnt = 0
for k, v in sorted(oov.items(), key=lambda x: -x[1]):
    if re.fullmatch(r'[a-zA-Z]+[.,?]?', k):
        continue
    print('{} => {}'.format(k, v))
    cnt += 1
    if cnt > 20:
        break

In [ ]:
oov.most_common(10)

In [ ]:
gc.collect()
time.sleep(2)

In [ ]:
super_frequent_words = ['a', 'of', 'to', 'and']
def split2(text):
    return (word for word in clean_text(text).split()
            if word not in super_frequent_words)

In [ ]:
sentences2 = train_all.question_text.progress_apply(split2).values
vocab2 = build_vocab(sentences2)

In [ ]:
oov2 = check_coverage(vocab2, embeddings_index)

In [ ]:
oov2.most_common(20)